In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def train_lstm_on_files(file_list, directory, input_columns, output_columns, batch_size=32, epochs=1):
    # Initialize the LSTM model
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(1, len(input_columns))),  # Adjust the input shape
        Dense(len(output_columns))  # Adjust the output layer based on the number of output columns
    ])
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    # Iterate over each file
    for file in file_list:
        file_path = os.path.join(directory, file)
        
        # Load the dataset from the current file
        df = pd.read_csv(file_path)
        
        # Preprocess the data for LSTM
        # Scale each input column and combine them
        scaler = StandardScaler()
        X_scaled = np.array([scaler.fit_transform(df[col].values.reshape(-1, 1)) for col in input_columns])
        X_scaled = np.transpose(X_scaled, (1, 0, 2)).reshape(-1, 1, len(input_columns))

        # Prepare output data
        y = df[output_columns].values

        # Split the data into training and validation
        X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        
        # Train the model on the data from the current file
        model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)
    
    # After training on all files, evaluate the model on the full dataset or a separate test set
    # Evaluation code goes here...

    # Return the trained model
    return model

# Example usage:
# file_list = ['batch1.csv', 'batch2.csv', ...]  # List of filenames
# directory = '/path/to/directory'
# input_columns = ['input1', 'input2', ...]  # List of input column names
# output_columns = ['output1', 'output2', ...]  # List of output column names
# model = train_lstm_on_files(file_list, directory, input_columns, output_columns)
